TCP Client–Server to Say “Hello”

In [ ]:
#hello_server.py

import socket

HOST = "0.0.0.0"  
PORT = 5000

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind((HOST, PORT))
server.listen(1)

print("Server waiting for connection...")

conn, addr = server.accept()
print("Connected to:", addr)

msg = conn.recv(1024).decode()
print("Client:", msg)

conn.send("Hello from Server!".encode())

conn.close()
server.close()


In [ ]:
#hello_client.py
import socket

HOST = "127.0.0.1"
PORT = 5000

client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect((HOST, PORT))

client.send("Hello from Client!".encode())

msg = client.recv(1024).decode()
print("Server:", msg)

client.close()


Peer-to-Peer File Transfer (TCP)

In [ ]:
#p2p_sender.py
import socket
import os

HOST = "0.0.0.0"
PORT = 6000

filename = "data.txt"  # put any file you want here

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind((HOST, PORT))
server.listen(1)

print("Waiting for peer...")

conn, addr = server.accept()
print("Connected to:", addr)

conn.send(filename.encode())
conn.recv(1024)  # acknowledgement

size = os.path.getsize(filename)
conn.send(str(size).encode())
conn.recv(1024)

with open(filename, "rb") as f:
    data = f.read()
    conn.sendall(data)

print("File sent.")
conn.close()
server.close()


In [ ]:
#p2p_receiver.py
import socket

HOST = "127.0.0.1"
PORT = 6000

client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect((HOST, PORT))

filename = client.recv(1024).decode()
client.send(b"OK")

size = int(client.recv(1024).decode())
client.send(b"OK")

data = b""
while len(data) < size:
    chunk = client.recv(1024)
    if not chunk:
        break
    data += chunk

with open("received_" + filename, "wb") as f:
    f.write(data)

print("File received:", "received_" + filename)
client.close()
